In [1]:
import ansys.fluent.core as pyfluent
import os
import glob
import openpyxl
from openpyxl.chart import LineChart, Reference
import csv
from moviepy.editor import ImageSequenceClip
os.environ['AWP_ROOT241']='C:\\Program Files\\ANSYS Inc\\v241'

c:\Users\austi\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
solver = pyfluent.launch_fluent(
	precision="single",
	processor_count=4,
	mode="solver",
)

solver.file.read_case(file_name=r'C:\Users\austi\OneDrive - Queensland University of Technology\cfd diagnosis\rw\\solved-fw-front.cas.h5')
solver.file.read_data(file_name=r'C:\Users\austi\OneDrive - Queensland University of Technology\cfd diagnosis\rw\\solved-fw-front.dat.h5')

Fast-loading "C:\PROGRA~1\ANSYSI~1\v241\fluent\fluent24.1.0\\addons\afd\lib\hdfio.bin"
Done.

Austins_Lenovo has other FLUENT parallel processes running.                       Process affinity not being set.

Reading from Austins_Lenovo:"C:\Users\austi\OneDrive - Queensland University of Technology\cfd diagnosis\rw\\solved-fw-front.cas.h5" in NODE0 mode ...
  Reading mesh ...
    16635232 cells,     1 cell zone  ...
       16635232 mixed cells,  zone id: 10056
    55674004 faces,     7 face zones ...
           1685 polygonal wall faces,  zone id: 6
         156088 polygonal wall faces,  zone id: 7
          93810 polygonal symmetry faces,  zone id: 5
          11624 polygonal pressure-outlet faces,  zone id: 4
           5532 polygonal symmetry faces,  zone id: 3
           1145 polygonal velocity-inlet faces,  zone id: 2
       55404120 mixed interior faces,  zone id: 10058
    22769470 nodes,     1 node zone  ...
         Combining every 32 partitions.
  Done.


Building...
     mes

In [3]:
#solver.tui.surface.plane_slice("plane_0m", 1, 0, 0, 0)
solver.tui.display.set.mirror_zones('symmetry')	

if not os.path.exists('C:\\Users\\austi\\OneDrive - Queensland University of Technology\\cfd diagnosis\\rw\\Contours_of_Velocity'):
    os.makedirs('C:\\Users\\austi\\OneDrive - Queensland University of Technology\\cfd diagnosis\\rw\\Contours_of_Velocity')
 
if not os.path.exists('C:\\Users\\austi\\OneDrive - Queensland University of Technology\\cfd diagnosis\\rw\\Contours_of_Static_Pressure'):
    os.makedirs('C:\\Users\\austi\\OneDrive - Queensland University of Technology\\cfd diagnosis\\rw\\Contours_of_Static_Pressure')

if not os.path.exists('C:\\Users\\austi\\OneDrive - Queensland University of Technology\\cfd diagnosis\\rw\\Contours_of_Total_Pressure'):
    os.makedirs('C:\\Users\\austi\\OneDrive - Queensland University of Technology\\cfd diagnosis\\rw\\Contours_of_Total_Pressure')
    
for i in range(0,40,1):
#round command used due to small rounding error if smaller step size is required changer round(x, 1) to >1
	x = -1+i*0.05
	count= round(x, 2)
	try:
		solver.tui.surface.delete_surface("plane_"+str(count)+"m")
	except:
		print('Passed')
	solver.tui.surface.plane_slice("plane_"+str(count)+"m", 1, 0, 0, count)
	plane = "plane_"+str(count)+"m"
	solver.results.graphics.contour["Contours_of_Velocity"+str(count)+"m"] = {
		"field": "velocity-magnitude",
		"surfaces_list": ["plane_"+str(count)+"m"],
		"node_values": True,
		"coloring": {
			"smooth": True,
		},
		"boundary_values": False,
		"range_option": {
			"option": "auto-range-off",
			"auto_range_off": {
				"clip_to_range": False,
				"minimum": 0,
				"maximum": 10, 
			},
		},
	}

	solver.results.graphics.contour["Contours_of_Static_Pressure"+str(count)+"m"] = {
		"field": "pressure",
		"surfaces_list": ["plane_"+str(count)+"m"],
		"node_values": True,
		"coloring": {
			"smooth": True,
		},
		"range_option": {
			"option": "auto-range-off",
			"auto_range_off": {
				"clip_to_range": False,
				"minimum": -100,
				"maximum": 100, 
			},		
		},
	}

	solver.results.graphics.contour["Contours_of_Total_Pressure"+str(count)+"m"] = {
		"field": "total-pressure",
		"surfaces_list": ["plane_"+str(count)+"m"],
		"node_values": True,
		"coloring": {
			"smooth": True,
		},
		"range_option": {
			"option": "auto-range-off",
			"auto_range_off": {
				"clip_to_range": False,
				"minimum": 0,
				"maximum": 50, 
			},		
		},
	}
	
	def makepicture(contour_type,scale):
		solver.results.graphics.views.restore_view(view_name="right")
		file_path='C:\\Users\\austi\\OneDrive - Queensland University of Technology\\cfd diagnosis\\rw\\'+ contour_type +'\\'
		solver.results.graphics.contour[contour_type+str(count)+"m"].display()
		solver.results.graphics.views.auto_scale()
		solver.results.graphics.views.camera.zoom(factor = scale)
		solver.results.graphics.picture.driver_options(hardcopy_format="jpeg")
		solver.results.graphics.picture(use_window_resolution = False)
		solver.results.graphics.picture(jpeg_hardcopy_quality=100)
		solver.results.graphics.picture.save_picture(file_name=file_path + contour_type+str(count)+"m.jpeg")
	
	makepicture('Contours_of_Velocity',2)
	makepicture('Contours_of_Static_Pressure',2)
	makepicture('Contours_of_Total_Pressure',2)
	solver.tui.surface.delete_surface("plane_"+str(count)+"m")

def convert_folder_to_video(contour_type, fps=2):
    """
    Converts a folder of PNG images to a video.

    Args:
        folder_path (str): Path to the folder containing PNG images.
        output_filename (str): Desired output video filename (e.g., "output.mp4").
        fps (int, optional): Frames per second for the video. Default is 2.
    """
    folder_path = "C:\\Users\\austi\\OneDrive - Queensland University of Technology\\cfd diagnosis\\rw\\" + contour_type + "\\"
    
    output_filename = folder_path + contour_type + '_sweep.mp4'
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".jpeg")]
    image_paths = [os.path.join(folder_path, img) for img in image_files]
    sorted_by_mtime_ascending = sorted(image_paths, key=lambda t: os.stat(t).st_mtime)
    clip = ImageSequenceClip(sorted_by_mtime_ascending, fps=fps)
    clip.write_videofile(output_filename, codec="libx264", audio = False)
    print(f"Video saved as {output_filename}")
    
if __name__ == "__main__":
    convert_folder_to_video('Contours_of_Velocity', 2)
    convert_folder_to_video('Contours_of_Static_Pressure', 2)
    convert_folder_to_video('Contours_of_Total_Pressure', 2)




ERROR: Invalid surface.
Error object: plane_-1.0m




Error: An error occurred in the previous operation.
       The details can be found in the text output.
Error Object: #f
Passed
The following solver settings object method could also be used to execute the above command:
<solver_session>.results.surfaces.plane_slice["plane_-1.0m"] = {}
solver.results.surfaces.plane_slice['plane_-1.0m'] = {"distance_from_origin" : -1.}


resulted in surface list selection changes
for the following graphics objects.
Contours_of_Total_Pressure-1.0m
Contours_of_Static_Pressure-1.0m
Contours_of_Velocity-1.0m




ERROR: Invalid surface.
Error object: plane_-0.95m




Error: An error occurred in the previous operation.
       The details can be found in the text output.
Error Object: #f
Passed
The following solver settings object method could also be used to execute the above command:
<solver_session>.results.surfaces.plane_slice["plane_-0.95m"] = {}
solver.results.surfaces.plane_slice['plane_-0.95m'] = 

Moviepy - Done !
Moviepy - video ready C:\Users\austi\OneDrive - Queensland University of Technology\cfd diagnosis\rw\Contours_of_Velocity\Contours_of_Velocity_sweep.mp4
Video saved as C:\Users\austi\OneDrive - Queensland University of Technology\cfd diagnosis\rw\Contours_of_Velocity\Contours_of_Velocity_sweep.mp4
Moviepy - Building video C:\Users\austi\OneDrive - Queensland University of Technology\cfd diagnosis\rw\Contours_of_Static_Pressure\Contours_of_Static_Pressure_sweep.mp4.
Moviepy - Writing video C:\Users\austi\OneDrive - Queensland University of Technology\cfd diagnosis\rw\Contours_of_Static_Pressure\Contours_of_Static_Pressure_sweep.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\austi\OneDrive - Queensland University of Technology\cfd diagnosis\rw\Contours_of_Static_Pressure\Contours_of_Static_Pressure_sweep.mp4
Video saved as C:\Users\austi\OneDrive - Queensland University of Technology\cfd diagnosis\rw\Contours_of_Static_Pressure\Contours_of_Static_Pressure_sweep.mp4
Moviepy - Building video C:\Users\austi\OneDrive - Queensland University of Technology\cfd diagnosis\rw\Contours_of_Total_Pressure\Contours_of_Total_Pressure_sweep.mp4.
Moviepy - Writing video C:\Users\austi\OneDrive - Queensland University of Technology\cfd diagnosis\rw\Contours_of_Total_Pressure\Contours_of_Total_Pressure_sweep.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\austi\OneDrive - Queensland University of Technology\cfd diagnosis\rw\Contours_of_Total_Pressure\Contours_of_Total_Pressure_sweep.mp4
Video saved as C:\Users\austi\OneDrive - Queensland University of Technology\cfd diagnosis\rw\Contours_of_Total_Pressure\Contours_of_Total_Pressure_sweep.mp4


In [4]:
# location="C:\\Users\\austi\\OneDrive - Queensland University of Technology\\cfd diagnosis\\rw\\"
location="C:\\Users\\austi\\cfd diagnosis\\rw\\"
solver.tui.report.forces.wall_forces(False, ['wing'], 0, 1, 0, True, "report.csv")                               
solver.tui.report.forces.wall_forces(False, ['wing'], 1, 0, 0, True, "report.csv")
solver.tui.report.forces.wall_forces(False, ['wing'], 0, 0, 1, True, "report.csv")
#solver.tui.report.projected_surface_area(False, ["wing"], 0.1489, 1, 0, 0, "projected_area.csv")
File_location = "C:\\Users\\austi\\OneDrive - Queensland University of Technology\\cfd diagnosis\\python\\"
Read_from = File_location + "report.csv"
Save_to = File_location + "Template.xlsx"

def read_specific_rows_to_excel(csv_file_path, row_numbers,cell_location,sheet):
    with open(csv_file_path, 'r') as file:
        reader = csv.reader(file)
        interesting_rows = [row for idx, row in enumerate(reader) if idx in row_numbers]
        my_string = ', '.join(map(str, interesting_rows)) #converting list to string
        my_list=my_string.split()
        half_value = float(my_list[1])
        value = half_value * 2
        workbook = openpyxl.load_workbook(Save_to)
        sheet = workbook[sheet]
        sheet[cell_location] = value
        workbook.save(Save_to)
# Example usage

row_numbers_to_read = [14] 
read_specific_rows_to_excel(Read_from, row_numbers_to_read, "AF26", "Sheet1")
row_numbers_to_read = [29] 
read_specific_rows_to_excel(Read_from, row_numbers_to_read, "AE26", "Sheet1")
'''
row_numbers_to_read = [x] 
read_specific_rows_to_excel(Read_from, row_numbers_to_read, "AE27", "Sheet1")
'''
workbook = openpyxl.load_workbook(Save_to)
sheet = workbook["Sheet1"]
sheet['A28'] = 'Basline images'
#creating image
location="C:\\Users\\austi\\OneDrive - Queensland University of Technology\\cfd diagnosis\\rw\\Contours_of_Static_Pressure\\Contours_of_Static_Pressure0.0m.jpeg"
img = openpyxl.drawing.image.Image(location)
img.height=305.5
img.width=405.8
#creates image and anchors next to defined cell
sheet.add_image(img,"A29")
workbook.save(Save_to)
def read_xy_file(file_path):
    x_values = []
    y_values = []
    with open(file_path, "r") as f:
        for line in f:
            try:
                x, y = map(float, line.split())
                x_values.append(x)
                y_values.append(y)
            except ValueError:
                # Skip lines that don't have valid x, y values
                pass
    return x_values, y_values

def create_and_save_graph(x_coords, y_coords):
    # Create a new Excel workbook
    workbook = openpyxl.load_workbook("C:\\Users\\austi\\OneDrive - Queensland University of Technology\\cfd diagnosis\\python\\Template.xlsx")
    sheet = workbook["Data"]
    #sheet.title = "Data"

    # Write data to the sheet
    for i, (x, y) in enumerate(zip(x_coords, y_coords), start=2):
        sheet[f"A{i}"] = x
        sheet[f"B{i}"] = y

    # Create a line chart
    chart = LineChart()
    chart.title = "Cumulative Drag Force"
    chart.x_axis.title = "Distance"
    chart.y_axis.title = "Drag Force"
    data = Reference(sheet, min_col=2, min_row=2, max_col=2, max_row=len(y_coords) + 1)
    categories = Reference(sheet, min_col=1, min_row=2, max_row=len(x_coords) + 1)
    chart.add_data(data, titles_from_data=True)
    chart.set_categories(categories)
    s1 = chart.series[0]
    s1.graphicalProperties.line.width = 50

    # Add the chart to the sheet
    sheet = workbook["Sheet1"]
    sheet.add_chart(chart, "H29")

    # Save the workbook
    workbook.save("C:\\Users\\austi\\OneDrive - Queensland University of Technology\\cfd diagnosis\\python\\Template.xlsx")

# Example usage:
xy_file_path = "C:\\Users\\austi\\Desktop\\xy_plot"  # Replace with the actual path to your .xy file
x_coords, y_coords = read_xy_file(xy_file_path)
create_and_save_graph(x_coords, y_coords)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\austi\\OneDrive - Queensland University of Technology\\cfd diagnosis\\python\\Template.xlsx'

In [14]:

solver.tui.plot.cumulative_plot( add ='drag_plot', zones='wing', number_of_divisions =200)
solver.tui.plot.cumulative_plot( add ='drag_plot', zones='wing', number_of_divisions =200)

TypeError: 'plot' object is not callable

In [ ]:
solver.tui.display.surface.line_surface("z", 0.01, 0.05, 0.05, -0.01, 0.05, 0.05)
solver.results.graphics.pathline["z"]= {
    "name" : "xx",
    "release_from_surfaces" : ["z"],
    "field" : "x-velocity",
    "step" : 500,
    "range": {
        "clip_to_range":{
            "min_value":0,
            "max_value":35,
        }
    }
}
file_path='C:\\Users\\austi\\OneDrive - Queensland University of Technology\\cfd diagnosis\\rw\\'
solver.results.graphics.pathline["z"].display()	
solver.results.graphics.views.auto_scale()
solver.results.graphics.views.camera.zoom(factor = 1)
solver.results.graphics.picture.driver_options(hardcopy_format="jpeg")
solver.results.graphics.picture(use_window_resolution = False)
solver.results.graphics.picture(jpeg_hardcopy_quality=100)
solver.results.graphics.picture.save_picture(file_name=file_path + "pathline.jpeg")

In [ ]:
''''''
from ansys.fluent.visualization.pyvista import  Graphics

graphics_session = Graphics(session)
pathlines1 = graphics_session.Pathlines["pathlines-1"]
pathlines1.field = "velocity-magnitude"
pathlines1.surfaces_list = ['inlet']
pathlines1.display("window-0")
''''''

ModuleNotFoundError: No module named 'ansys.fluent.visualization'

In [ ]:

solver.tui.display.surface.line_surface("test2", 0.01, 0.05, 0.05, -0.01, 0.05, 0.05)
pathline: solver.tui.display.objects.create(
    "pathlines",
    "pathlines-1",
    "field",
    "time",
    "accuracy-control",
    "tolerance",
    "0.001",
    "skip",
    "5",
    "surfaces-list",
    "inlet-1",
    "inlet-2",
    "inlet-3",
    "()",
    "quit",
)
create contour on wing
take pictures
create scene
export avz